1. Importe os pacotes necessários:
- O Numpy é usado para armazenar todos os dados, recuperá-los do modelo e trabalhar com eles.

- O OpenCV é usado para ler quadros de um arquivo de vídeo ou da transmissão da webcam, redimensioná-los e remodelá-los de acordo com os requisitos do modelo. Ele também fornece o módulo dnn, que usaremos para trabalhar com nosso modelo de rede neural profunda. Além disso, ele desenha as caixas delimitadoras na imagem e as mostra ao usuário.

- O Os é usado para trabalhar com arquivos, ler caminhos e outras operações relacionadas a arquivos.

- O Imutils é outra ótima biblioteca para realizar diferentes ações em imagens. Ele atua como uma biblioteca auxiliar fornecendo algumas funções muito úteis para o OpenCV, que já é uma biblioteca vasta.

In [1]:
import numpy as np
import cv2
import os
import imutils

https://data-flair.training/blogs/pedestrian-detection-python-opencv/

https://github.com/adityagandhamal/tensorflow-object-detection

Declarando algumas variáveis de limiar que usaremos posteriormente. O limiar NMS é o limite para separar previsões sobrepostas, mais sobre NMS adiante. MinConfiança é o limite para a pontuação de confiança retornada pelo modelo acima do qual uma previsão é considerada verdadeira.

In [2]:
NMS_THRESHOLD=0.3
MIN_CONFIDENCE=0.2

2. Função de Detecção de Pedestres

Agora vamos definir a função mais importante que usaremos para a detecção de pedestres. Ela recebe o quadro da imagem, quadro a quadro, do OpenCV, seja da webcam ou de um arquivo de vídeo, o modelo, o nome da camada de saída da qual obteremos o resultado e uma variável "personidz".
Obtemos as dimensões do quadro passado e inicializamos uma matriz que armazenará o resultado do modelo.

In [3]:
def pedestrian_detection(image, model, layer_name, personidz=0):
	(H, W) = image.shape[:2]
	results = []

	# Construímos um blob com o quadro que recebemos e, em seguida, o passamos para o modelo YOLO, executando uma passagem direta 
 	# (forward pass), que retornará as caixas delimitadoras das detecções e as pontuações de confiança para cada detecção. O model.forward 
 	# retornará a saída da camada que foi passada para ele, ou seja, a camada de saída.
	blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
	model.setInput(blob)
	layerOutputs = model.forward(layer_name)

	# Crie arrays para armazenar as caixas delimitadoras resultantes, centroides e as pontuações de confiança associadas.
	boxes = []
	centroids = []
	confidences = []

	# layerOutputs é uma lista de listas que contém as saídas. Cada lista em layerOutputs contém detalhes sobre uma única 
 	# previsão, como a caixa delimitadora, a confiança, etc. Portanto, todas as previsões estão agrupadas em uma lista de listas. 
 	# Percorremos essa estrutura usando um loop.
	for output in layerOutputs:
		for detection in output:

			# A partir da detecção individual, obtemos as pontuações para todas as previsões, o ID da classe e a confiança para o 
   			# ID da classe com pontuação máxima.
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]
   
   
			# Obtendo a Detecção
			# Agora precisamos obter a detecção que desejamos. O ID da classe para detectar uma pessoa é 0 no formato YOLO, portanto, 
   			# verificamos isso e também se a confiança está acima do limite para evitar falsos positivos.
			# O YOLO retorna o centróide da caixa delimitadora com a altura e largura da caixa, em vez da coordenada superior esquerda da caixa.
			if classID == personidz and confidence > MIN_CONFIDENCE:

				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				# Agora, como não temos as coordenadas do canto superior direito da caixa delimitadora, vamos calculá-las subtraindo 
    			# metade da largura e altura do ponto x do centróide e ponto y do centróide, respectivamente.
				# Agora que temos todas as informações de que precisamos, vamos adicioná-las às listas que criamos anteriormente.
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))
    
	
	# Out of the loop here we do nms or non maxima suppression to remove overlapping and weak bounding boxes. Because neighboring 
 	# windows of the actual detection also contain high scores, hundreds of predictions pop up that we need to clean. We use the opencv’s 
 	# dnn module provided nmsboxes function for this
	idzs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONFIDENCE, NMS_THRESHOLD)
 
	# Verifique se temos alguma detecção e percorra-a. Extraia as coordenadas da caixa delimitadora, a largura e altura, 
 	# e adicione todos os detalhes à nossa lista de resultados que criamos anteriormente. Por fim, retorne essa lista e encerre a função.
	if len(idzs) > 0:
		for i in idzs.flatten():
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])
			res = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(res)
   
   
	return results

In [10]:
# Aqui carregamos os rótulos nos quais o modelo foi treinado. Ele possui 80 rótulos de classes no arquivo coco.names no formato COCO.
# Defina o caminho para os arquivos de pesos (weights) e de configuração (cfg) para o nosso modelo. Lembre-se de que esses são caminhos
# relativos e, neste caso, como os arquivos estão na mesma pasta, usamos apenas o nome como caminhos.

labelsPath = "names/coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

weights_path = "yolo/yolov4-tiny.weights"
config_path = "yolo/yolov4-tiny.cfg"

model = cv2.dnn.readNetFromDarknet(config_path, weights_path)

# If your Opencvdnn compiled with cuda and you want to run this on the GPU uncomment these lines to use cuda enabled GPUs.
model.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
model.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Precisamos do nome da última camada de aprendizado profundo para obter a saída dela. Em seguida, inicializamos o VideoCapture do OpenCV.
# Se você deseja usar a webcam, passe 0 em vez do nome do arquivo para a função VideoCapture.
layer_name = model.getLayerNames()
layer_name = [layer_name[i - 1] for i in model.getUnconnectedOutLayers()]
cap = cv2.VideoCapture("videos/Test_2.mp4")
writer = None

1. Lendo os quadros e processando-os
- Agora, inicie um loop infinito e leia os quadros do VideoCapture. Se os quadros acabarem, ou seja, o vídeo terminar, saímos do loop infinito.

- Redimensione a imagem de acordo com os requisitos sem alterar a proporção da imagem usando a função resize do imutils.

- Passe a imagem, o nome da camada de saída do modelo e o ID da classe para a etiqueta "person" para a função pedestrian_detection que criamos anteriormente.

In [11]:
while True:
	(grabbed, image) = cap.read()

	if not grabbed:
		break
	image = imutils.resize(image, width=700)
	results = pedestrian_detection(image, model, layer_name,
		personidz=LABELS.index("person"))

	# Obtendo os resultados da função, percorremos os resultados e desenhamos retângulos de caixa delimitadora nas previsões e mostramos 
 	# ao usuário. Capturamos a tecla ESC pressionada com a função waitKey do OpenCV e saímos do loop. Por fim, liberamos a captura e fechamos 
 	# todas as janelas.
	for res in results:
		cv2.rectangle(image, (res[1][0],res[1][1]), (res[1][2],res[1][3]), (0, 255, 0), 2)

	cv2.imshow("Detection",image)

	key = cv2.waitKey(1)
	if key == 27:
		break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@371.576] global net_impl.cpp:174 setUpNet DNN module was not built with CUDA backend; switching to CPU


# Hands Position Detection

In [3]:
# Install the Libraries


import cv2
import mediapipe as mp
import time
cap = cv2.VideoCapture("videos/hands.mp4")
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
cTime = 0
pTime = 0
# Function Start
while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    #print(results.multi_hand_landmarks)
    if results.multi_hand_landmarks:
        for handlms in results.multi_hand_landmarks:
            for id, lm in enumerate(handlms.landmark):
                #print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                # print(id, cx, cy)
                #if id == 5:
                cv2.circle(img, (cx, cy), 15, (139, 0, 0), cv2.FILLED)


            mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
# Time and FPS Calculation

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    
    cv2.putText(img, str(int(fps)), (10,70), cv2.FONT_HERSHEY_SIMPLEX, 3, (139,0,0), 3)

    cv2.imshow("Image", img)
    cv2.waitKey(1)

error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


: 